# 🏆 Portfolio Momentum: Winner Takes All Strategy

Questa strategia implementa una logica di **momentum sui portafogli**: ogni venerdì, investe al 100% nel portafoglio che ha performato meglio (risk-adjusted) nella settimana appena conclusa.

- **Tre portafogli candidati**: Commodities contrarian, Forex contrarian, TLT
- **Selezione settimanale**: Winner-takes-all basato su Sharpe ratio settimanale
- **No lookahead bias**: Decisione presa venerdì, applicata dal lunedì
- **Visualizzazioni**: Equity curve + evoluzione pesi nel tempo

---

## 📦 Setup e Importazioni

In [ ]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Importa moduli core
from core.commodity_momentum import (
    generate_long_short_commodity_momentum_signals,
    calculate_portfolio_performance
)
from core.data_loader import load_forex_data
from core.signal_generator import generate_momentum_signals
from core.backtest_engine import BacktestEngine

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
print('✅ Setup completato!')

## 📊 Caricamento Dati e Configurazione

- Carichiamo i dati per commodities, forex e TLT
- Impostiamo i parametri delle strategie contrarian

In [ ]:
# Parametri strategia
lookback_commodities = 30
commodities_top_n = 3
lookback_forex = 30
forex_top_n = 3
rebalance_weekday = 4  # venerdì

print(f"Parametri strategia:")
print(f"- Commodities: lookback {lookback_commodities}d, top {commodities_top_n}")
print(f"- Forex: lookback {lookback_forex}d, top {forex_top_n}")
print(f"- Rebalancing: Venerdì (weekday {rebalance_weekday})")

# Caricamento dati
commodities_df = pd.read_parquet('../data/commodities_extended_data.parquet')
forex_df = load_forex_data('../data/forex_extended_data.parquet')

print(f'\n✅ Dati caricati:')
print(f'✅ Commodities: {commodities_df.shape[0]} giorni, {commodities_df.shape[1]} colonne')
print(f'✅ Forex: shape da BacktestEngine')

## 🔄 Generazione Segnali Contrarian e Portfolio

- Generiamo i segnali contrarian per commodities e forex
- Calcoliamo le equity curve dei tre portafogli candidati

In [ ]:
# Segnali contrarian commodities
signals_momentum_commodities = generate_long_short_commodity_momentum_signals(
    commodities_df, lookback_days=lookback_commodities, top_n=commodities_top_n, rebalance_weekday=rebalance_weekday
)
signals_contrarian_commodities = signals_momentum_commodities * -1

# Segnali contrarian forex
signals_momentum_forex = generate_momentum_signals(
    price_data=forex_df,
    lookback_days=lookback_forex,
    top_n=forex_top_n,
    rebalance_freq='weekly'
)
signals_contrarian_forex = signals_momentum_forex * -1

print('✅ Segnali contrarian generati per commodities e forex!')

## 🚀 Backtest Strategie e Caricamento TLT

- Eseguiamo il backtest delle strategie contrarian
- Carichiamo i dati TLT per completare i tre portafogli candidati

In [ ]:
# Backtest commodities contrarian
contrarian_equity_commodities = calculate_portfolio_performance(signals_contrarian_commodities, commodities_df)
contrarian_returns_commodities = contrarian_equity_commodities.pct_change().fillna(0)

# Backtest forex contrarian
engine_contrarian_forex = BacktestEngine(initial_capital=100000, leverage=1.0)
results_contrarian_forex = engine_contrarian_forex.run_backtest(forex_df, signals_contrarian_forex, verbose=False)
contrarian_equity_forex = engine_contrarian_forex.portfolio_value
contrarian_returns_forex = engine_contrarian_forex.portfolio_returns

# Caricamento dati TLT
commodities_start = contrarian_equity_commodities.index[0]
commodities_end = contrarian_equity_commodities.index[-1]
forex_start = contrarian_equity_forex.index[0] 
forex_end = contrarian_equity_forex.index[-1]

tlt_start = min(commodities_start, forex_start)
tlt_end = max(commodities_end, forex_end)

print(f"Scaricando TLT dal {tlt_start.strftime('%Y-%m-%d')} al {tlt_end.strftime('%Y-%m-%d')}")

tlt_data = yf.download('TLT', start=tlt_start, end=tlt_end, progress=False)
if 'Adj Close' in tlt_data.columns:
    tlt_prices = tlt_data['Adj Close']
else:
    tlt_prices = tlt_data['Close']

if tlt_prices.index.tz:
    tlt_prices.index = tlt_prices.index.tz_localize(None)

tlt_equity = tlt_prices / tlt_prices.iloc[0]
tlt_returns = tlt_equity.pct_change().fillna(0)

print(f"✅ TLT scaricato: {len(tlt_prices)} giorni dal {tlt_prices.index[0]} al {tlt_prices.index[-1]}")
print('✅ Backtest completato per tutte le strategie!')

## 📐 Allineamento Dati e Preparazione Portfolio

- Allineiamo le tre equity curve alle date comuni
- Normalizziamo tutte le curve a partire da 1.0

In [ ]:
# Rimuovi i fusi orari dagli indici per allineare le date
commodities_idx = contrarian_equity_commodities.index.tz_localize(None) if contrarian_equity_commodities.index.tz else contrarian_equity_commodities.index
forex_idx = contrarian_equity_forex.index.tz_localize(None) if contrarian_equity_forex.index.tz else contrarian_equity_forex.index
tlt_idx = tlt_equity.index.tz_localize(None) if tlt_equity.index.tz else tlt_equity.index

commodities_eq = pd.Series(contrarian_equity_commodities.values, index=commodities_idx)
forex_eq = pd.Series(contrarian_equity_forex.values, index=forex_idx)

# Fix per TLT: gestione dimensionalità
if isinstance(tlt_equity, pd.Series):
    tlt_eq = tlt_equity.copy()
    tlt_eq.index = tlt_idx
else:
    tlt_values = tlt_equity.values.flatten() if tlt_equity.values.ndim > 1 else tlt_equity.values
    tlt_eq = pd.Series(tlt_values, index=tlt_idx)

# Trova il periodo comune tra tutti e tre gli asset
start_date = max(commodities_eq.index[0], forex_eq.index[0], tlt_eq.index[0])
end_date = min(commodities_eq.index[-1], forex_eq.index[-1], tlt_eq.index[-1])

print(f"Periodo combinato: {start_date} → {end_date}")

# Filtra per il periodo comune
commodities_eq = commodities_eq.loc[start_date:end_date]
forex_eq = forex_eq.loc[start_date:end_date]
tlt_eq = tlt_eq.loc[start_date:end_date]

# Allinea le date (intersection di tutti e tre)
common_dates = commodities_eq.index.intersection(forex_eq.index).intersection(tlt_eq.index)
print(f"Date comuni trovate: {len(common_dates)}")

commodities_eq = commodities_eq.loc[common_dates]
forex_eq = forex_eq.loc[common_dates]
tlt_eq = tlt_eq.loc[common_dates]

# Normalizza tutte le equity curve a 1 all'inizio
commodities_eq_norm = commodities_eq / commodities_eq.iloc[0]
forex_eq_norm = forex_eq / forex_eq.iloc[0]
tlt_eq_norm = tlt_eq / tlt_eq.iloc[0]

print("✅ Tre portafogli allineati:")
print(f"✅ Commodities: {len(commodities_eq_norm)} giorni")
print(f"✅ Forex: {len(forex_eq_norm)} giorni")
print(f"✅ TLT: {len(tlt_eq_norm)} giorni")
print(f"✅ Periodo: {common_dates[0].strftime('%Y-%m-%d')} → {common_dates[-1].strftime('%Y-%m-%d')}")

## 🏆 Winner-Takes-All Portfolio Momentum Strategy

- **Logica**: Ogni venerdì, investi al 100% nel portafoglio con miglior Sharpe ratio settimanale
- **Metrica**: Risk-adjusted return (rendimento settimanale / volatilità rolling 30d)
- **Applicazione**: Dal lunedì successivo per tutta la settimana (no lookahead bias)

In [ ]:
# Crea DataFrame con le tre equity curve per facilità di calcolo
portfolios_df = pd.DataFrame({
    'Commodities': commodities_eq_norm,
    'Forex': forex_eq_norm,
    'TLT': tlt_eq_norm
}, index=common_dates)

# Calcola i rendimenti giornalieri
portfolios_returns = portfolios_df.pct_change().fillna(0)

# Parametri strategia
volatility_window = 30  # giorni per rolling volatility
risk_free_rate = 0.02   # 2% annuale per Sharpe ratio

print(f"Portfolio DataFrame creato: {portfolios_df.shape}")
print("Primi 5 giorni:")
print(portfolios_df.head())

# Trova tutti i venerdì per il rebalancing
rebalance_dates = common_dates[common_dates.weekday == 4]  # Venerdì
print(f"\n✅ Date di rebalancing (venerdì): {len(rebalance_dates)}")
print(f"Prima: {rebalance_dates[0].strftime('%Y-%m-%d')}, Ultima: {rebalance_dates[-1].strftime('%Y-%m-%d')}")

In [ ]:
# Implementazione algoritmo Winner-Takes-All
def calculate_risk_adjusted_performance(returns_series, window=5):
    """
    Calcola performance risk-adjusted per una finestra di giorni.
    Usa rendimento medio / volatilità come metrica.
    """
    if len(returns_series) < window:
        return np.nan
    
    weekly_returns = returns_series.tail(window)
    mean_return = weekly_returns.mean()
    volatility = weekly_returns.std()
    
    if volatility == 0 or np.isnan(volatility):
        return mean_return  # Se no volatilità, usa solo il rendimento
    
    return mean_return / volatility

# Dizionari per tracciare la strategia
selected_portfolio = {}  # {date: 'portfolio_name'}
performance_metrics = {}  # {date: {'Commodities': score, 'Forex': score, 'TLT': score}}
weekly_allocations = []  # Lista per tracking evoluzione pesi

print("🔍 Implementando algoritmo di selezione settimanale...")

# Loop attraverso tutte le date di rebalancing
for i, rebalance_date in enumerate(rebalance_dates):
    if i == 0:
        # Prima settimana: inizia con TLT (asset più stabile)
        winner = 'TLT'
        print(f"Prima settimana ({rebalance_date.strftime('%Y-%m-%d')}): Inizia con {winner}")
    else:
        # Calcola performance della settimana appena conclusa (5 giorni lavorativi)
        # Trova la settimana precedente (da lunedì a venerdì)
        prev_week_end = rebalance_date
        prev_week_start = prev_week_end - pd.Timedelta(days=4)  # 5 giorni lavorativi
        
        # Filtra solo i giorni lavorativi nella settimana precedente
        week_dates = common_dates[(common_dates >= prev_week_start) & (common_dates <= prev_week_end)]
        
        if len(week_dates) >= 3:  # Almeno 3 giorni per calcolo affidabile
            scores = {}
            for portfolio_name in portfolios_returns.columns:
                week_returns = portfolios_returns.loc[week_dates, portfolio_name]
                scores[portfolio_name] = calculate_risk_adjusted_performance(week_returns, window=len(week_returns))
            
            # Seleziona il portfolio con score più alto
            winner = max(scores, key=scores.get)
            performance_metrics[rebalance_date] = scores
            
            if i <= 5:  # Mostra primi 5 per debug
                print(f"{rebalance_date.strftime('%Y-%m-%d')}: Scores = {scores}, Winner = {winner}")
        else:
            # Non abbastanza dati, mantieni selezione precedente
            winner = list(selected_portfolio.values())[-1] if selected_portfolio else 'TLT'
    
    selected_portfolio[rebalance_date] = winner
    
    # Trova la data di applicazione (lunedì successivo)
    application_start = rebalance_date + pd.Timedelta(days=1)
    while application_start.weekday() > 4:  # Skip weekend
        application_start += pd.Timedelta(days=1)
    
    # Trova la fine della settimana di applicazione
    next_rebalance_idx = i + 1
    if next_rebalance_idx < len(rebalance_dates):
        application_end = rebalance_dates[next_rebalance_idx]
    else:
        application_end = common_dates[-1]
    
    # Registra allocazione per la settimana
    weekly_allocations.append({
        'rebalance_date': rebalance_date,
        'application_start': application_start,
        'application_end': application_end,
        'selected_portfolio': winner,
        'Commodities': 1.0 if winner == 'Commodities' else 0.0,
        'Forex': 1.0 if winner == 'Forex' else 0.0,
        'TLT': 1.0 if winner == 'TLT' else 0.0
    })

weekly_allocations_df = pd.DataFrame(weekly_allocations)
print(f"\n✅ Algoritmo completato: {len(selected_portfolio)} decisioni di rebalancing")
print("✅ Prime 3 selezioni:")
for i, (date, portfolio) in enumerate(list(selected_portfolio.items())[:3]):
    print(f"  {date.strftime('%Y-%m-%d')}: {portfolio}")

In [ ]:
# Costruzione equity curve della strategia Winner-Takes-All
print("📈 Costruendo equity curve della strategia momentum...")

# Crea serie temporale con allocazioni giornaliere
daily_allocations = pd.DataFrame(index=common_dates, columns=['Commodities', 'Forex', 'TLT'])
daily_allocations = daily_allocations.fillna(0.0)

# Applica le allocazioni settimanali
for allocation in weekly_allocations:
    start_date = allocation['application_start']
    end_date = allocation['application_end']
    selected = allocation['selected_portfolio']
    
    # Trova le date effettive nell'intervallo
    mask = (daily_allocations.index >= start_date) & (daily_allocations.index <= end_date)
    application_dates = daily_allocations.index[mask]
    
    if len(application_dates) > 0:
        # Imposta allocazione al 100% per il portafoglio selezionato
        daily_allocations.loc[application_dates, :] = 0.0
        daily_allocations.loc[application_dates, selected] = 1.0

# Verifica allocazioni
print("Controllo allocazioni:")
print(f"Somma allocazioni (dovrebbe essere sempre 1.0): {daily_allocations.sum(axis=1).describe()}")
print("Prime 10 allocazioni:")
print(daily_allocations.head(10))

# Calcola equity curve della strategia momentum
momentum_portfolio_values = []

for date in common_dates:
    if date in daily_allocations.index:
        weights = daily_allocations.loc[date]
        portfolio_value = (
            weights['Commodities'] * commodities_eq_norm.loc[date] +
            weights['Forex'] * forex_eq_norm.loc[date] +
            weights['TLT'] * tlt_eq_norm.loc[date]
        )
        momentum_portfolio_values.append(portfolio_value)
    else:
        # Fallback: usa valore precedente o 1.0
        momentum_portfolio_values.append(momentum_portfolio_values[-1] if momentum_portfolio_values else 1.0)

momentum_equity = pd.Series(momentum_portfolio_values, index=common_dates)
momentum_returns = momentum_equity.pct_change().fillna(0)

print(f"\n✅ Strategia momentum costruita:")
print(f"✅ Equity finale: {momentum_equity.iloc[-1]:.3f}")
print(f"✅ Rendimento totale: {(momentum_equity.iloc[-1] - 1) * 100:.2f}%")
print(f"✅ Rendimento annualizzato: {((momentum_equity.iloc[-1] ** (252 / len(momentum_equity))) - 1) * 100:.2f}%")

## 📊 Visualizzazioni: Equity Curve e Evoluzione Pesi

- **Chart 1**: Confronto equity curve strategia momentum vs portafogli individuali
- **Chart 2**: Evoluzione pesi nel tempo - quale portafoglio è selezionato ogni settimana

In [ ]:
# Grafico 1: Equity Curves Comparison
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Top chart: Equity curves
ax1.plot(commodities_eq_norm.index, commodities_eq_norm.values, 
         label='Commodities Contrarian', color='orange', linewidth=2, alpha=0.7)
ax1.plot(forex_eq_norm.index, forex_eq_norm.values, 
         label='Forex Contrarian', color='green', linewidth=2, alpha=0.7)
ax1.plot(tlt_eq_norm.index, tlt_eq_norm.values, 
         label='TLT', color='purple', linewidth=2, alpha=0.7)
ax1.plot(momentum_equity.index, momentum_equity.values, 
         label='🏆 Winner-Takes-All Momentum', color='red', linewidth=3)

# Equal weight benchmark
equal_weight_equity = (commodities_eq_norm + forex_eq_norm + tlt_eq_norm) / 3
ax1.plot(equal_weight_equity.index, equal_weight_equity.values, 
         label='Equal Weight (33% each)', color='black', linewidth=2, linestyle='--', alpha=0.6)

ax1.set_title('🏆 Portfolio Momentum Strategy vs Individual Portfolios', fontsize=16, fontweight='bold')
ax1.set_ylabel('Equity (normalized to 1.0)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Bottom chart: Weight Evolution
# Crea grafico a area stackata per mostrare l'evoluzione dei pesi
colors = ['orange', 'green', 'purple']
ax2.fill_between(daily_allocations.index, 0, daily_allocations['Commodities'], 
                 alpha=0.7, color=colors[0], label='Commodities')
ax2.fill_between(daily_allocations.index, daily_allocations['Commodities'], 
                 daily_allocations['Commodities'] + daily_allocations['Forex'],
                 alpha=0.7, color=colors[1], label='Forex')
ax2.fill_between(daily_allocations.index, 
                 daily_allocations['Commodities'] + daily_allocations['Forex'],
                 daily_allocations['Commodities'] + daily_allocations['Forex'] + daily_allocations['TLT'],
                 alpha=0.7, color=colors[2], label='TLT')

ax2.set_title('📊 Portfolio Allocation Evolution (Winner-Takes-All)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Allocation Weight')
ax2.set_xlabel('Date')
ax2.set_ylim(0, 1)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizzazioni create!")

## 📈 Analisi Performance e Metriche Comparative

- **Confronto**: Strategia momentum vs portafogli individuali vs equal weight
- **Metriche**: Sharpe, Sortino, Max Drawdown, Calmar Ratio, Volatilità

In [ ]:
# Funzione per calcolare metriche di performance
def calculate_performance_metrics(equity_series, returns_series):
    """
    Calcola un set completo di metriche di performance.
    """
    metrics = {}
    
    # Basic returns
    total_return = (equity_series.iloc[-1] - 1) * 100
    annualized_return = ((equity_series.iloc[-1] ** (252 / len(equity_series))) - 1) * 100
    
    # Volatility
    volatility = returns_series.std() * np.sqrt(252) * 100
    
    # Sharpe ratio (assume 2% risk-free rate)
    excess_returns = returns_series - (0.02 / 252)
    sharpe = excess_returns.mean() / returns_series.std() * np.sqrt(252) if returns_series.std() > 0 else 0
    
    # Sortino ratio (downside deviation)
    downside_returns = returns_series[returns_series < 0]
    downside_dev = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino = excess_returns.mean() / (downside_dev / np.sqrt(252)) if downside_dev > 0 else 0
    
    # Maximum drawdown
    rolling_max = equity_series.cummax()
    drawdown = (equity_series / rolling_max - 1) * 100
    max_drawdown = drawdown.min()
    
    # Calmar ratio
    calmar = annualized_return / abs(max_drawdown) if max_drawdown != 0 else 0
    
    # Win rate
    win_rate = len(returns_series[returns_series > 0]) / len(returns_series) * 100
    
    return {
        'Total Return (%)': total_return,
        'Annualized Return (%)': annualized_return,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Max Drawdown (%)': max_drawdown,
        'Calmar Ratio': calmar,
        'Win Rate (%)': win_rate
    }

# Calcola metriche per tutte le strategie
strategies = {
    'Winner-Takes-All': (momentum_equity, momentum_returns),
    'Equal Weight': (equal_weight_equity, equal_weight_equity.pct_change().fillna(0)),
    'Commodities Only': (commodities_eq_norm, commodities_eq_norm.pct_change().fillna(0)),
    'Forex Only': (forex_eq_norm, forex_eq_norm.pct_change().fillna(0)),
    'TLT Only': (tlt_eq_norm, tlt_eq_norm.pct_change().fillna(0))
}

performance_results = {}
for strategy_name, (equity, returns) in strategies.items():
    performance_results[strategy_name] = calculate_performance_metrics(equity, returns)

# Crea DataFrame per visualizzazione
performance_df = pd.DataFrame(performance_results).T

print('📊 PERFORMANCE COMPARISON - WINNER TAKES ALL STRATEGY')
print('=' * 90)
print(performance_df.round(2))
print('=' * 90)

# Evidenzia i migliori risultati per categoria
print('\n🏆 BEST PERFORMERS PER METRICA:')
for metric in performance_df.columns:
    if 'Drawdown' in metric:
        best = performance_df[metric].idxmax()  # Less negative is better for drawdown
    else:
        best = performance_df[metric].idxmax()
    print(f'{metric:<25}: {best} ({performance_df.loc[best, metric]:.2f})')

## 🎯 Analisi Frequenza Selezioni e Statistiche Avanzate

- **Selection Frequency**: Quante volte ogni portafoglio è stato selezionato\n- **Streak Analysis**: Periodi consecutivi di selezione dello stesso portafoglio\n- **Turnover**: Frequenza di cambio portafoglio

In [ ]:
# Analisi frequenza selezioni
selection_counts = weekly_allocations_df['selected_portfolio'].value_counts()
selection_percentages = (selection_counts / len(weekly_allocations_df) * 100).round(1)

print('📊 ANALISI FREQUENZA SELEZIONI')
print('=' * 50)
for portfolio, count in selection_counts.items():
    percentage = selection_percentages[portfolio]
    print(f'{portfolio:<15}: {count:3d} settimane ({percentage:5.1f}%)')
print('=' * 50)

# Streak Analysis
def analyze_streaks(selections):
    """
    Analizza i periodi consecutivi di selezione dello stesso portafoglio.
    """
    streaks = []
    current_portfolio = None
    current_streak = 0
    
    for portfolio in selections:
        if portfolio == current_portfolio:
            current_streak += 1
        else:
            if current_streak > 0:
                streaks.append((current_portfolio, current_streak))
            current_portfolio = portfolio
            current_streak = 1
    
    # Aggiungi l'ultimo streak
    if current_streak > 0:
        streaks.append((current_portfolio, current_streak))
    
    return streaks

streaks = analyze_streaks(weekly_allocations_df['selected_portfolio'].tolist())

print('\n🔥 STREAK ANALYSIS (Periodi consecutivi)')
print('=' * 50)
print(f'Total streaks: {len(streaks)}')

# Raggruppa per portafoglio
streak_stats = {}
for portfolio, length in streaks:
    if portfolio not in streak_stats:
        streak_stats[portfolio] = []
    streak_stats[portfolio].append(length)

for portfolio in ['Commodities', 'Forex', 'TLT']:
    if portfolio in streak_stats:
        lengths = streak_stats[portfolio]
        avg_streak = np.mean(lengths)
        max_streak = max(lengths)
        print(f'{portfolio:<15}: Avg {avg_streak:.1f} settimane, Max {max_streak} settimane')
    else:
        print(f'{portfolio:<15}: Nessun periodo selezionato')

# Turnover analysis
changes = 0
for i in range(1, len(weekly_allocations_df)):
    if weekly_allocations_df.iloc[i]['selected_portfolio'] != weekly_allocations_df.iloc[i-1]['selected_portfolio']:
        changes += 1

turnover_rate = changes / (len(weekly_allocations_df) - 1) * 100

print(f'\n💱 TURNOVER ANALYSIS')
print('=' * 50)
print(f'Portfolio changes: {changes} su {len(weekly_allocations_df)-1} possibili')
print(f'Turnover rate: {turnover_rate:.1f}%')
print(f'Avg holding period: {(100/turnover_rate):.1f} settimane' if turnover_rate > 0 else 'N/A')

# Visualizzazione grafico a torta delle selezioni
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart delle selezioni
colors_pie = ['orange', 'green', 'purple']
wedges, texts, autotexts = ax1.pie(selection_counts.values, labels=selection_counts.index, 
                                   autopct='%1.1f%%', colors=colors_pie, startangle=90)
ax1.set_title('📊 Portfolio Selection Frequency\n(Winner-Takes-All Strategy)', fontweight='bold')

# Bar chart dei turnover per periodo
# Calcola turnover rolling su finestre di 52 settimane
if len(weekly_allocations_df) >= 52:
    rolling_turnover = []
    window = 52
    
    for i in range(window, len(weekly_allocations_df)):
        window_data = weekly_allocations_df.iloc[i-window:i]
        window_changes = 0
        for j in range(1, len(window_data)):
            if window_data.iloc[j]['selected_portfolio'] != window_data.iloc[j-1]['selected_portfolio']:
                window_changes += 1
        rolling_turnover.append(window_changes / (window-1) * 100)
    
    rolling_dates = weekly_allocations_df['rebalance_date'].iloc[window:]
    ax2.plot(rolling_dates, rolling_turnover, color='red', linewidth=2)
    ax2.set_title('📈 Rolling Turnover Rate (52-week window)', fontweight='bold')
    ax2.set_ylabel('Turnover Rate (%)')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'Not enough data\nfor rolling turnover', 
             ha='center', va='center', transform=ax2.transAxes, fontsize=12)
    ax2.set_title('Rolling Turnover Rate', fontweight='bold')

plt.tight_layout()
plt.show()

print('\n✅ Analisi completa terminata!')